In [1]:
from my_parser import *
import sys

In [19]:
calculator = Parser()
calculator.add_bnf('num','?=[0-9]+/*s:int(s)*/')
calculator.add_bnf('rterm',"$(num)|'('$(sum)')'")
calculator.add_bnf('term',
'''$(rterm) 
|?= ${t:rterm} '*' ${n:term} /* t,n:t*n */ 
|?= ${t:rterm} '/' ${n:term} /* t,n:t/n */ 
''')
calculator.add_bnf('sum',
'''$(term)
|?= ${t:term} '+' ${n:sum} /* t,n:t+n */ 
|?= ${t:term} '-' ${n:sum} /* t,n:t-n */ 
''')
calculator.main = 'sum'

In [20]:
calculator.parse('2+2*2')

6

In [23]:
calculator.debug = False
calculator.parse('(2+2)*2')

8

In [25]:
calculator.debug = False
calculator.parse('2+2+2+2')

8

In [9]:
calculator.p_bnf_sequence_('(?t=term) `*` (?n=rterm) /* t,n:t*n */',0)

(10,
 ('obj',
  <function my_parser.make_sequence.<locals>.pp_implicit_obj_join(s, p0)>))

In [10]:
calculator.p_bnf_sequence_('?= (?t=term) `*` (?n=rterm) /* t,n:t*n */',0)

(13,
 ('direct',
  <function my_parser.make_sequence.<locals>.pp_direct_obj_join(s, p0)>))